In [2]:
import cv2
import numpy as np
import time
from datetime import datetime
import os
import threading
import pywhatkit
from queue import Queue
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Constants
LINE_Y = 200  # in pixels from the top
WHATSAPP = True
DELETE_SAVED_IMAGES = True
TIME_INTERVAL = 1.5 * 60  # 1.5 minutes in seconds
WHATSAPP_NUMBER = "+91919191191"
IMAGE_SEND_DELAY = 30  # 30 seconds between sending images

# Global variables
last_save_time = time.time()
image_queue = Queue()
sending_lock = threading.Lock()

def saveImage(frame, x, y, w, h, timestamp):
    try:
        if w < 50 or h < 50:
            return

        # Ensure we don't go out of frame bounds
        y_start = max(y-100, 0)
        y_end = min(y+h+100, frame.shape[0])
        x_start = max(x-100, 0)
        x_end = min(x+w+100, frame.shape[1])
        
        face = frame[y_start:y_end, x_start:x_end]
        if face.size == 0:
            return

        face = cv2.putText(face, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                          0.5, (0, 255, 255), 2, cv2.LINE_AA)
        
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        current_time_formatted = datetime.now().strftime("on %B %d %Y, at %I:%M:%S %p")
        
        # Create saved directory if it doesn't exist
        save_dir = "C:/Users/Mayur/OneDrive/Pictures/saved"
        os.makedirs(save_dir, exist_ok=True)
        
        face_path = os.path.join(save_dir, f"face_{current_time}.jpg")
        cv2.imwrite(face_path, face)
        
        print(f"✅ Face saved at {current_time}")
        print(f"📁 Save location: {face_path}")

        # Add to queue instead of list
        image_queue.put((face_path, current_time_formatted))
        print(f"📤 Image queued for sending to WhatsApp")
        
    except Exception as e:
        print(f"❌ Error saving image: {e}")

def send_whatsapp_image():
    while True:
        try:
            if not image_queue.empty():
                with sending_lock:
                    face_path, timestamp = image_queue.get()
                    if os.path.exists(face_path):
                        try:
                            print(f"🔄 Preparing to send image: {face_path}")
                            pywhatkit.sendwhats_image(
                                WHATSAPP_NUMBER,
                                face_path,
                                f"Face detected {timestamp}"
                            )
                            print(f"✅ Image sent: {face_path}")
                            
                            if DELETE_SAVED_IMAGES:
                                os.remove(face_path)
                                print(f"🗑️ Image deleted: {face_path}")
                            
                            print(f"⏳ Waiting {IMAGE_SEND_DELAY} seconds before sending next image...")
                            time.sleep(IMAGE_SEND_DELAY)
                            print("✅ Ready for next image")
                        except Exception as e:
                            print(f"❌ Error sending image: {e}")
                            print("🔄 Requeueing failed image for retry")
                            image_queue.put((face_path, timestamp))
            else:
                # Sleep briefly when queue is empty to prevent CPU spinning
                time.sleep(1)
        except Exception as e:
            print(f"❌ Error in send_whatsapp_image thread: {e}")
            time.sleep(1)

def drawRectangles(frame, faces):
    new_faces = []
    for i, (x, y, w, h) in enumerate(faces):
        if w > 50 and h > 50:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"Face {i+1}", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            new_faces.append((x, y, w, h))
    
    faces = np.array(new_faces)
    cv2.putText(frame, f"No. of faces: {len(faces)}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return faces

def checkLineCrossing(faces, frame, frame_copy):
    global last_save_time
    for (x, y, w, h) in faces:
        if y < LINE_Y:
            cv2.putText(frame, "WARNING: Face crossed line!", (10, 720 - 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            if time.time() - last_save_time >= 3:
                saveImage(frame_copy, x, y, w, h, time.ctime())
                last_save_time = time.time()

def main():
    try:
        print("🚀 Starting Face Detection System")
        print(f"👤 WhatsApp number configured: {WHATSAPP_NUMBER}")
        print(f"⏱️ Image send delay: {IMAGE_SEND_DELAY} seconds")
        
        # Start WhatsApp sending thread
        whatsapp_thread = threading.Thread(target=send_whatsapp_image, daemon=True)
        whatsapp_thread.start()
        print("✅ WhatsApp sending thread started")
        
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        print("✅ Camera initialized")

        face_cascade = cv2.CascadeClassifier(
            "D:/downloads/SpyEye-main/SpyEye-main/XML_Files/haarcascade_frontalface_default.xml")
        print("✅ Face detection cascade loaded")

        print("⚡ System ready - Press 'q' to quit")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to grab frame")
                break

            frame_copy = frame.copy()
            cv2.line(frame, pt1=(0, LINE_Y), pt2=(1280, LINE_Y),
                     color=(255, 255, 255), thickness=2)
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
            
            faces = drawRectangles(frame, faces)
            checkLineCrossing(faces, frame, frame_copy)
            
            cv2.imshow("Face Recognition", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                print("👋 Shutting down system...")
                break

    except Exception as e:
        print(f"❌ Error in main: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("✅ System shutdown complete")

if __name__ == "__main__":
    main()


🚀 Starting Face Detection System
👤 WhatsApp number configured: +919130207210
⏱️ Image send delay: 30 seconds
✅ WhatsApp sending thread started
✅ Camera initialized
✅ Face detection cascade loaded
⚡ System ready - Press 'q' to quit
✅ Face saved at 2025-01-20_14-12-19
📁 Save location: C:/Users/Mayur/OneDrive/Pictures/saved\face_2025-01-20_14-12-19.jpg
📤 Image queued for sending to WhatsApp
✅ Face saved at 2025-01-20_14-12-28
📁 Save location: C:/Users/Mayur/OneDrive/Pictures/saved\face_2025-01-20_14-12-28.jpg
📤 Image queued for sending to WhatsApp
✅ Face saved at 2025-01-20_14-12-37
📁 Save location: C:/Users/Mayur/OneDrive/Pictures/saved\face_2025-01-20_14-12-37.jpg
📤 Image queued for sending to WhatsApp
✅ Face saved at 2025-01-20_14-12-40
📁 Save location: C:/Users/Mayur/OneDrive/Pictures/saved\face_2025-01-20_14-12-40.jpg
📤 Image queued for sending to WhatsApp
✅ Face saved at 2025-01-20_14-12-43
📁 Save location: C:/Users/Mayur/OneDrive/Pictures/saved\face_2025-01-20_14-12-43.jpg
📤 Image 